In [6]:
from pathlib import Path
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from config import Config_list

# accuracy list from experiments test
%store -r acc_list


In [7]:
# restart kernel after running config.py
len(Config_list)

4

In [8]:
from pathlib import Path
import matplotlib.pyplot as plt

# create df and label target label new
df_list = []
csv_list = []
for i in range(len(Config_list)):
    
    if Config_list[i][1]['TYPE'] == "BINARY":
        if Config_list[i][1]["DATA_SET"] not in csv_list:
            temp_df = pd.read_csv(Config_list[i][1]["DATA_SET"])
            csv_list.append(Config_list[i][1]["DATA_SET"])
            temp_df = temp_df.drop(columns=Config_list[i][1]['DROP'])
            temp_df= temp_df.rename(columns={temp_df[Config_list[i][1]['TARGET_LABEL']].name : 'TARGET_LABEL'})
            df_list.append(temp_df)
        
eval_df = pd.DataFrame(csv_list, columns=['Data_Set'])
eval_df["Target_1_pos"] = pd.Series('int32')
eval_df["Target_2_neg"] = pd.Series('int32')
eval_df["IB_Ratio"] = pd.Series()
eval_df["Num_Feautures"] = pd.Series('int32')
eval_df["Cat_Feautures"] = pd.Series('int32')
eval_df["Size_row"] = pd.Series('int32')
eval_df["Size_col"] = pd.Series('int32')
eval_df["BRCG_Recall"] = pd.Series()

eval_df

C:\Users\hornd\AppData\Local\R-MINI~1\envs\aix360i_v2\lib\site-packages\ipykernel_launcher.py:20: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
C:\Users\hornd\AppData\Local\R-MINI~1\envs\aix360i_v2\lib\site-packages\ipykernel_launcher.py:25: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.


,Data_Set,Target_1_pos,Target_2_neg,IB_Ratio,Num_Feautures,Cat_Feautures,Size_row,Size_col,BRCG_Recall
0,../data/german_credit_codiert.csv,int32,int32,NaN,int32,int32,int32,int32,NaN
1,../data/fraud_detection.csv,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,../data/TaiwanCreditData.csv,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,../data/miniloan-decisions-100K.csv,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
# iterate over the datasets and get needed metrics
for frame in range(len(df_list)):
    metric = df_list[frame]["TARGET_LABEL"].value_counts()

    # Imbalanced Ratio = minor class
    if metric[0] > metric[1]:
        eval_df["Target_1_pos"].iloc[frame] = metric[1]
        eval_df["Target_2_neg"].iloc[frame] = metric[0]
    else:
        eval_df["Target_1_pos"].iloc[frame] = metric[0]
        eval_df["Target_2_neg"].iloc[frame] = metric[1]
        
    df_size_row = len(df_list[frame])
    df_size_col = len(df_list[frame].columns)
    df_num_feauture =  len(df_list[frame].select_dtypes(include=['int64', 'float64']).columns)
    df_cat_feauture =   len(df_list[frame].select_dtypes(include=['object']).columns)

    eval_df["IB_Ratio"].iloc[frame] = eval_df["Target_1_pos"].iloc[frame]/eval_df["Target_2_neg"].iloc[frame]
    eval_df["Size_row"].iloc[frame]   = df_size_row
    eval_df["Size_col"].iloc[frame]   = df_size_col
    
    eval_df["Num_Feautures"].iloc[frame] = df_num_feauture
    eval_df["Cat_Feautures"].iloc[frame] = df_cat_feauture
    #eval_df["BRCG_Recall"].iloc[frame] = acc_list[frame]


eval_df

C:\Users\hornd\AppData\Local\R-MINI~1\envs\aix360i_v2\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


,Data_Set,Target_1_pos,Target_2_neg,IB_Ratio,Num_Feautures,Cat_Feautures,Size_row,Size_col,BRCG_Recall
0,../data/german_credit_codiert.csv,300,700,0.428571,8,13,1000,21,NaN
1,../data/fraud_detection.csv,492,284315,0.001730,31,0,284807,31,NaN
2,../data/TaiwanCreditData.csv,6636,23364,0.284027,15,9,30000,24,NaN
3,../data/miniloan-decisions-100K.csv,21954,78046,0.281296,6,1,100000,8,NaN


In [10]:
eval_df.to_csv("test.csv");